In [ ]:
!pip install gensim konlpy
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
!bash Mecab-ko-for-Google-Colab/install_mecab-ko_on_colab190912.sh

     |████████████████████████████████| 19.4 MB 841 kB/s 
     |████████████████████████████████| 86 kB 7.2 MB/s 
     |████████████████████████████████| 448 kB 58.7 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 91, done.
remote: Total 91 (delta 0), reused 0 (delta 0), pack-reused 91
Unpacking objects: 100% (91/91), done.
Installing konlpy.....
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2021-12-12 08:54:44--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::22c0:3470, 2406:da00:ff00::3403:4be7, ...
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP reque

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.cuda as cuda
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from torch.utils.data import DataLoader, Dataset
from google.colab import drive
from konlpy.tag import Mecab
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split

In [ ]:
device = 'cuda' if cuda.is_available() else 'cpu'
print(f'Training MNIST Model on {device}\n{"="*44}')

drive.mount('/content/drive/')

Training MNIST Model on cuda
Mounted at /content/drive/


In [ ]:
df = pd.read_csv("/content/drive/Shareddrives/실전기계_project/data/data_spo_aug_sr_train_1210.csv")
#df = pd.read_csv("/content/drive/Shareddrives/실전기계_project/data/data_spo_aug_rs_train_1210.csv")
#df = pd.read_csv("/content/drive/Shareddrives/실전기계_project/data/data_spo_aug_ri_train_1210.csv")
#df = pd.read_csv("/content/drive/Shareddrives/실전기계_project/data/data_spo_aug_rd_train_1210.csv")
#df = pd.read_csv("/content/drive/Shareddrives/실전기계_project/data/data_spo_aug_all_train_1210.csv")
#df = pd.read_csv("/content/drive/Shareddrives/실전기계_project/data/data_spo_aug_X_train_1210.csv")
df

,Unnamed: 0,content,label
0,1427,꽤 볼만한 스릴러. 반전에 약간 실망할 수도.,1
1,439,손으로 셀수없을만큼 여러번 봤어도 늘 같은 장면에서 눈물이 난다. 김양이 엄마 찾을...,1
2,1537,할복할 자신도 없으면서 푼돈이나 뜯으려는 찌질이...모토메의 사연을 몰랐을 땐 우리...,1
3,1334,애초에 천재설정 영화에서 감독의허황이라고 비판하는 사람들이 더 이해안감;;,0
4,53,"젊을때의 그들, 어설프지만 풋풋한 모습.",0
...,...,...,...
3097,503,레드퍼드는 위대한 개츠비에도 그렇고 마지막에 뒤에서 총 맞는 장면이 여기서 있네,1
3098,846,딱한마디만하겠습니다....... 최고!!!,0
3099,1269,영화관에 가서 보면 더 좋았을걸...어린시절로 돌아간 느낌이들어 만감이...,0
3100,512,세번째 아이 낳기 전에 아버지랑 마지막으로 만나 산책하는 장면에서 눈물...주인공이...,1


## For Down Sampling



In [ ]:
# If needed down sampling
np.random.seed(42)

df_0 = df[df['label'] == 0]
df_1 = df[df['label'] == 1]

min_len = min(len(df_0), len(df_1))

df = pd.concat([df_0[:min_len], df_1[:min_len]])
df = df.sample(frac=1, random_state=42)

train_df = df
test_df = pd.read_csv("/content/drive/Shareddrives/실전기계_project/data/data_spo_test_1211.csv")

TRAIN_SET_LENGTH = len(train_df)
TEST_SET_LENGTH = len(test_df)

train_df

,Unnamed: 0,content,label
1168,913,이젠 시간여행을 하지않는다.내가 이날을 위 상 여행을 한 것처럼살아간다 이렇게 말했...,1
875,925,인어공주의 사랑이야기가 신선하고 흥미로웠다.,0
570,103,"마지막 무지개다리를 건너,이든과 만낫을때 폭풍오열..우리아이와도 그렇게 만나게되겟죠",1
1743,13,화려하게 정성들인 제사상이 왤케 슬프죠??ㅠㅠ 눈물샘 폭발 꺼이꺼이임지호님 어머니곁...,1
1095,753,오페라 좋아하는 사람을 이해할수가 없었는데 이 영화의 나를 울게하소서 이 하나만으로...,0
...,...,...,...
3079,610,시즌1에서는 미국 스타일의 키 큰 여자가 최종회에서 죽었다면 최상의 전,1
2173,1049,정말 멋진 결말!ㅎ,0
2233,378,느낌이 오래도록 가는 영화... 사와지리 에리카는 정말 세 하나는 잘 보는듯.,0
2584,109,벌써 마지막회라니 믿기싫은현실ㅜ 넘 재밌어요,0


In [ ]:
document = []
mecab = Mecab()

# document tokenize
for doc in train_df['content']:
  document.append(mecab.morphs(doc))

In [ ]:
class TextDataset(Dataset):
  def __init__(self, data, target, length, transform=None):
    super(TextDataset, self).__init__()

    self.data = data
    self.target = target
    self.length = length

  def __len__(self):
    return len(self.target)

  def __getitem__(self, idx):
    return self.data[idx], self.target[idx], self.length[idx]

## Hyperparameters

In [ ]:
#BATCH_SIZES = [32]  # no arg
BATCH_SIZES = [64]  # arg
INPUT_SIZES = [32, 64]
HIDDEN_SIZES = [64, 128]
LEARNING_RATES = [1e-3, 3e-3, 5e-3]
WEIGHTDECAY = 0.9
EPOCH = 40
TRY = 3
SEED = 42

## Train

In [ ]:
def train_test(BATCH_SIZE, INPUT_SIZE, HIDDEN_SIZE, LR, TRY):
  global train_df, test_df, document, WEIGHTDECAY

  tr_a, tr_l, te_a, te_l = None, None, None, None

  # word2vec train
  model = Word2Vec(document, size=INPUT_SIZE, window=5, min_count=1, sg=1, workers=2)

  def make_tensor(df):
    word_arr = [mecab.morphs(text) for text in df['content']]
    data = [torch.Tensor([model.wv[word] for word in words if word in model.wv]) for words in word_arr]
    
    # for debug when test data words not in word2vec dictionary
    #for i, d in enumerate(data):
    #  if len(d) == 0:
    #    print(df.loc[i, 'content'])
    
    doc_length = torch.LongTensor(list(map(lambda x: len(x), data)))
    target = torch.FloatTensor(df['label'].to_numpy())
    data = pad_sequence(data, batch_first=True)
      
    return data, doc_length, target

  train_data, train_doc_length, train_target = make_tensor(train_df)
  test_data, test_doc_length, test_target = make_tensor(test_df)

  train_dataloader = DataLoader(TextDataset(train_data, train_target, train_doc_length), batch_size=BATCH_SIZE, shuffle=True)
  test_dataloader = DataLoader(TextDataset(test_data, test_target, test_doc_length), batch_size=BATCH_SIZE, shuffle=True)

  class RNN(nn.Module):
    def __init__(self):
      super(RNN, self).__init__()
      self.lstm = nn.RNN(input_size=INPUT_SIZE, hidden_size=HIDDEN_SIZE, num_layers=2, bidirectional=True, dropout=0.3, batch_first=True)
      self.linear_1 = nn.Linear(HIDDEN_SIZE * 2, 64)
      self.linear_2 = nn.Linear(64, 1)
      self.batchnorm_1 = nn.BatchNorm1d(64)
      self.relu = nn.ReLU()
      self.sigmoid = nn.Sigmoid()

    def forward(self, x, length):
      packed_input = pack_padded_sequence(x, length.tolist(), batch_first=True, enforce_sorted=False)
      packed_outputs, hidden = self.lstm(packed_input)
      output, output_length = pad_packed_sequence(packed_outputs, batch_first=True)
      last_seq_idxs = torch.LongTensor([x - 1 for x in output_length])
      output = output[range(output.shape[0]), last_seq_idxs, :]
      x = self.batchnorm_1(self.relu(self.linear_1(output)))
      x = self.sigmoid(self.linear_2(x))
      return x.view(-1)

  import os
  os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

  model = RNN()
  model.to(device)

  def weight_init_kaiming_normal(submodule):
    if isinstance(submodule, torch.nn.RNN) or isinstance(submodule, torch.nn.LSTM):
      for name, param in submodule.named_parameters():
        if 'weight_ih' in name:
            torch.nn.init.xavier_uniform_(param.data)
        elif 'weight_hh' in name:
            torch.nn.init.orthogonal_(param.data)
        elif 'bias' in name:
            param.data.fill_(0)
    elif isinstance(submodule, torch.nn.Linear):
      torch.nn.init.kaiming_normal_(submodule.weight)
      submodule.bias.data.fill_(0.01)
    elif isinstance(submodule, torch.nn.BatchNorm1d):
      submodule.weight.data.fill_(1.0)
      submodule.bias.data.zero_()
  
  model.apply(weight_init_kaiming_normal)

  criterion = F.binary_cross_entropy
  optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHTDECAY)

  scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, [8, 16, 24, 32], gamma=0.5)

  # train
  for epoch in range(EPOCH):
    corrent_results_sum = 0.0
    avarage_loss = 0.0
    for i, data in enumerate(train_dataloader):
      inputs, labels, length = data
      inputs, labels, length = inputs.to(device), labels.to(device), length
      y_pred = model(inputs, length)
      loss = criterion(y_pred, labels, reduction='mean')

      y_pred_tag = torch.round(y_pred)
      corrent_results_sum += (y_pred_tag == labels).sum().float()
      avarage_loss += loss.item()

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      scheduler.step()
    
    #if epoch % 10 == 0:
      #print(f"Epoch: {epoch + 1} | Acc: {corrent_results_sum * 100 / TRAIN_SET_LENGTH} | Loss: {avarage_loss / len(train_dataloader)}")
    
    if epoch == EPOCH - 1:
      tr_a, tr_l = corrent_results_sum * 100 / TRAIN_SET_LENGTH, avarage_loss / len(train_dataloader)
      print(f"Try: {TRY} | Train Acc: {corrent_results_sum * 100 / TRAIN_SET_LENGTH} | Train Loss: {avarage_loss / len(train_dataloader)}")


  # test
  corrent_results_sum = 0.0
  avarage_loss = 0.0
  for i, data in enumerate(test_dataloader):
    inputs, labels, length = data
    inputs, labels, length = inputs.to(device), labels.to(device), length
    y_pred = model(inputs, length)
    loss = criterion(y_pred, labels, reduction='mean')

    y_pred_tag = torch.round(y_pred)
    corrent_results_sum += (y_pred_tag == labels).sum().float()
    avarage_loss += loss.item()

  te_a, te_l = corrent_results_sum * 100 / TEST_SET_LENGTH, avarage_loss / len(test_dataloader)
  print(f"Try: {TRY} | Test Acc: {corrent_results_sum * 100 / TEST_SET_LENGTH} | Test Loss: {avarage_loss / len(test_dataloader)}")

  return tr_a, tr_l, te_a, te_l

In [ ]:
for batch_size in BATCH_SIZES:
  for input_size in INPUT_SIZES:
    for hidden_size in HIDDEN_SIZES:
      for lr in LEARNING_RATES:
        train_acc, train_loss, test_acc, test_loss = 0.0, 0.0, 0.0, 0.0
        print(f"Case => Batch Size: {batch_size} | Input Size: {input_size} | Hidden Size: {hidden_size} | Learning Rate: {lr}")
        for t in range(1, TRY + 1):
          tr_a, tr_l, te_a, te_l = train_test(batch_size, input_size, hidden_size, lr, t)
          train_acc += tr_a
          train_loss += tr_l
          test_acc += te_a
          test_loss += te_l
        train_acc /= TRY
        train_loss /= TRY
        test_acc /= TRY
        test_loss /= TRY
        print(f"Total Average => Train Acc: {train_acc} | Train Loss: {train_loss} | Test Acc: {test_acc} | Test Loss: {test_loss}")
        print("")

Case => Batch Size: 64 | Input Size: 32 | Hidden Size: 64 | Learning Rate: 0.001
Try: 1 | Train Acc: 85.3642807006836 | Train Loss: 0.3536658068092502
Try: 1 | Test Acc: 70.45454406738281 | Test Loss: 0.6407657265663147
Try: 2 | Train Acc: 84.46163177490234 | Train Loss: 0.35896322861009716
Try: 2 | Test Acc: 68.63636016845703 | Test Loss: 0.6189238876104355
Try: 3 | Train Acc: 83.5267562866211 | Train Loss: 0.3711549274775447
Try: 3 | Test Acc: 69.54545593261719 | Test Loss: 0.6593561619520187
Total Average => Train Acc: 84.45088958740234 | Train Loss: 0.36126132096563074 | Test Acc: 69.54545593261719 | Test Loss: 0.6396819253762563

Case => Batch Size: 64 | Input Size: 32 | Hidden Size: 64 | Learning Rate: 0.003
Try: 1 | Train Acc: 88.2333984375 | Train Loss: 0.27520874811678514
Try: 1 | Test Acc: 70.90908813476562 | Test Loss: 0.7317611873149872
Try: 2 | Train Acc: 89.45841217041016 | Train Loss: 0.24951863106416197
Try: 2 | Test Acc: 68.18181610107422 | Test Loss: 0.713129878044128

In [ ]:
8